In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

In [4]:
class pUser(BaseModel):
  name: str
  age: int
  email: str

In [5]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [6]:
foo_p.name

'Jane'

In [7]:
# this code is expected to fail
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing

In [8]:
class Class(BaseModel):
  students: List[pUser]

In [9]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [10]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

#### Pydantic to OpenAI function

Docstring for the class is required as it is converted to description field of the functions object

In [12]:
class WeatherSearch(BaseModel):
  """Call this with an airport code to get the weather at that airport"""
  airport_code: str = Field(description="airport code to get weather for")

In [13]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [14]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

/Users/ojaskapre/projects/notebooks/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [15]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [16]:
from langchain.chat_models import ChatOpenAI

In [17]:
model = ChatOpenAI()

/Users/ojaskapre/projects/notebooks/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [21]:
# passing functions in invoke

model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 70, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-e722c7a2-87f4-478c-8aef-006d3264d318-0')

In [22]:
# binding the model with function and then invoking the new model

model_with_function = model.bind(functions=[weather_function])
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-eb4d7e79-b11c-460e-b913-18a76791e123-0')

##### Forcing the function call

In [23]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [24]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 79, 'total_tokens': 86}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-881be552-e4e5-46dc-8bd5-64962ad176e1-0')

In [25]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 74, 'total_tokens': 81}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c00cd680-535f-4725-aacb-680167d3032e-0')

##### Using it in a chain

In [26]:
from langchain.prompts import ChatPromptTemplate

In [27]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [28]:
chain = prompt | model_with_function

In [29]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 75, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7e930aad-18dc-4438-970a-a814c7e5e7b7-0')

#### Using multiple functions

In [30]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [31]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [32]:
functions

[{'name': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
     'type': 'string'}},
   'required': ['airport_code'],
   'type': 'object'}},
 {'name': 'ArtistSearch',
  'description': 'Call this to get the names of songs by a particular artist',
  'parameters': {'properties': {'artist_name': {'description': 'name of artist to look up',
     'type': 'string'},
    'n': {'description': 'number of results', 'type': 'integer'}},
   'required': ['artist_name', 'n'],
   'type': 'object'}}]

In [33]:
model_with_functions = model.bind(functions=functions)

In [34]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 116, 'total_tokens': 133}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-81fb5e3d-761d-40e6-86ef-1adb8ad8e61c-0')

In [35]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 118, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f611fc93-6f88-48cd-82a7-ebda0ad1b076-0')

In [36]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e618b411-87dc-4c86-b00b-3e0fcc533d27-0')